# **Setup**

In [ ]:
!nvidia-smi

Tue Nov 10 21:35:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile 
# specifying the zip file name 
file_name = "/content/drive/My Drive/images_fullsize.zip"
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zi: 
    # extracting all the files 
    print('Extracting all the files now...') 
    zi.extractall() 
    print('Done!')

Extracting all the files now...
Done!


In [ ]:
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torchvision import transforms
import torch
import torchvision
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim.lr_scheduler import OneCycleLR
import pandas as pd 
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm_notebook as tqdm 
from sklearn.model_selection import train_test_split

In [ ]:
import random
import numpy as np
SEED_VAL  = 42
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED):
  random.seed(SEED_VAL)
  np.random.seed(SEED_VAL)
  torch.manual_seed(SEED_VAL)
  torch.cuda.manual_seed_all(SEED_VAL)
  os.environ['PYTHONHASHSEED'] = str(SEED_VAL)
  torch.backends.cudnn.deterministic = True

# **Utilities** 

In [ ]:
import cv2
def default_image_loader(path):
    return Image.open(path).convert('RGB')

def cv_reader(path):
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

class ImageDataset(Dataset):
    def __init__(self, data_path, df, transform=None,mode='train'):
        self.df = df
        self.loader = cv_reader
        self.transform = transform
        self.dir = data_path
        self.mode = mode

    def __getitem__(self, index):
        image_name = self.df.ID[index]
        image = self.loader(os.path.join(self.dir, image_name+'.png'))
        if self.transform is not None:
          image = self.transform(image=image)
          image=image["image"]
        

        if self.mode == 'train':
            label = self.df.LABEL[index]
            return {'image' : torch.tensor(image,dtype=torch.float), 
                'label' : torch.tensor(label,dtype = torch.float) }
            
        return {'image' : torch.tensor(image,dtype=torch.float), 
}            
        
    
    def __len__(self):
        return self.df.shape[0]

In [ ]:
class Net(nn.Module):
    def __init__(self,name):
        super(Net, self).__init__()
        self.name  = name
        
        
        if name == 'densenet201':
          self.arch = models.densenet201(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,2,bias=True)
        

          
    def forward(self, x):
        """
        """
        x = self.arch(x)
        
        return x

In [ ]:
class AverageMeter():
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
def loss_fn(outputs,targets):
  criterion = nn.BCELoss()
  outputs = nn.Softmax()(outputs)[:,1]
  loss = criterion(outputs,targets)
  return loss 

In [ ]:
def train_fn(train_data_loader,model,optimizer,device,scheduler = None):
  model.train()
  losses = AverageMeter()
  roc_auc = AverageMeter()
  tk0 = tqdm(train_data_loader, total=len(train_data_loader))
  tot_loss = 0
  for bi,d in enumerate(tk0):
    images = d['image']
    labels = d['label']

    #send them to device 
    images = images.to(device,dtype=torch.float)
    labels = labels.to(device,dtype=torch.float)
    optimizer.zero_grad()
    if BOOL_CROP:
      bs, ncrops, c, h, w = images.size()
      outputs = model(images.view(-1, c, h, w)) # fuse batch size and ncrops
      outputs = outputs.view(bs, ncrops, -1).mean(1) # avg over crops
    
    else:
      outputs  = model(images)
      
    loss = loss_fn(outputs,labels)
    loss.backward()
    optimizer.step()
    tot_loss = tot_loss + loss.item()

    #calculate roc_auc score for val 
    outputs = outputs[:,1]
    roc_auc.update(roc_auc_score(labels.cpu().numpy(),outputs.cpu().detach().numpy()))
    losses.update(loss.item(), labels.size(0))
    tk0.set_postfix(loss=losses.avg,roc_auc_scoree=roc_auc.avg)


    if scheduler is not None:
      scheduler.step()

  mse_score = tot_loss/len(train_data_loader)
  roc_auc_scoree = roc_auc_score(labels.cpu().numpy(),outputs.cpu().detach().numpy())
  
  print("Training loss for this epoch: ", mse_score)
  print("Training roc_auc_score for this epoch: ", roc_auc_scoree)
  return roc_auc_scoree

In [ ]:
def eval_fn(valid_data_loader,model,device):
  model.eval()
  tot_loss = 0
  final_outputs = []
  final_targets = []
  with torch.no_grad():
    for bi,d in enumerate(valid_data_loader):

      images = d['image']
      labels = d['label']

      #send them to device 
      images = images.to(device,dtype=torch.float)
      labels = labels.to(device,dtype=torch.float)
      if BOOL_CROP:
        bs, ncrops, c, h, w = images.size()
        outputs = model(images.view(-1, c, h, w)) # fuse batch size and ncrops
        outputs = outputs.view(bs, ncrops, -1).mean(1) # avg over crops
      else:
        outputs  = model(images)

      loss = loss_fn(outputs,labels)
      tot_loss = tot_loss + loss.item()

      
      final_outputs.append(outputs.cpu().detach().numpy())
      final_targets.append(labels.cpu().numpy())


    final_targets = np.concatenate(final_targets)
    final_outputs = np.concatenate(final_outputs)
    
    loss_score  = tot_loss/len(valid_data_loader)
    roc_auc_scoree = roc_auc_score(final_targets,final_outputs[:,1])
    print("Validation loss for this epoch: ",loss_score)
    print('Validation roc_auc_score for this epoch',roc_auc_scoree)
  return roc_auc_scoree

In [ ]:
import albumentations as A
#from albumentations.pytorch.transforms import ToTensorV2
from albumentations.pytorch.transforms import ToTensor
from albumentations import torch as albtorch

In [ ]:
input_shape = [312,256]

def get_transforms():
  train_transform = A.Compose([
                              
                              A.Resize(height=input_shape[0], width=input_shape[1],p=1),
                              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                              albtorch.ToTensor(),
                               
                            ])
  val_transform = A.Compose([
                              A.Resize(height=input_shape[0], width=input_shape[1],p=1),
                              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                              albtorch.ToTensor(),
                               
                            ])
  return train_transform,val_transform

## **Training**

In [ ]:
import warnings 
warnings.simplefilter('ignore')

In [ ]:
NAME = 'densenet201'
EPOCHS = 15
TRAIN_BATH_SIZE = 16
BOOL_CROP = False
skf = StratifiedKFold(n_splits=5,random_state=SEED_VAL)
all_rmse_scores = []
def run_folds():
  seed_all(SEED_VAL)
  train_transform,val_transform = get_transforms()
  train = pd.read_csv('/content/Train.csv')
  DF = pd.DataFrame()
  for i,(train_index,val_index) in enumerate(skf.split(train,y=train.LABEL)):
    print(f"#########################  Fold {i+1}/{skf.n_splits}  #########################")
    train_df , valid_df = train.iloc[train_index,:],train.iloc[val_index,:]
    train_df = train_df.reset_index()
    valid_df = valid_df.reset_index()
    train_dataset = ImageDataset('/content/train/',train_df,train_transform)
    valid_dataset = ImageDataset('/content/train/',valid_df,val_transform)
    train_data_loader = DataLoader(dataset=train_dataset,shuffle=True,batch_size=TRAIN_BATH_SIZE)
    valid_data_loader = DataLoader(dataset=valid_dataset,shuffle=False,batch_size=32)
    device = torch.device("cuda")
    model = Net(NAME)
    model.to(device)

    num_train_steps = int(len(train_df) /TRAIN_BATH_SIZE * EPOCHS )
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    #scheduler = MultiStepLR(optimizer, milestones=[2,5,10], gamma=0.1)
    best_roc = 0.5
    DF = pd.DataFrame()
    for epoch in range(EPOCHS):
      print("----------------EPOCH "+str(epoch+1)+"---------------------")
      AUC_train = train_fn(train_data_loader, model, optimizer, device,scheduler=None)
      AUC_val = eval_fn(valid_data_loader ,model, device)
      DF = DF.append({'EPOCHS':1+epoch,'TRAIN_AUC':AUC_train,'VAL_AUC':AUC_val},ignore_index=True)
      if AUC_val>best_roc:
        best_roc = AUC_val 
        torch.save(model.state_dict(),f"best_model_{i}") 
    print('best AUC_val: ',best_roc)
  return DF

In [ ]:
kfold_results = run_folds()

#########################  Fold 1/5  #########################
----------------EPOCH 1---------------------



Training loss for this epoch:  0.47096911983357537
Training roc_auc_score for this epoch:  0.8571428571428571
Validation loss for this epoch:  0.3085077226161957
Validation roc_auc_score for this epoch 0.950028940767895
----------------EPOCH 2---------------------



Training loss for this epoch:  0.17931777776943314
Training roc_auc_score for this epoch:  1.0
Validation loss for this epoch:  0.2505944438278675
Validation roc_auc_score for this epoch 0.9523442021994983
----------------EPOCH 3---------------------



Training loss for this epoch:  0.0951270865690377
Training roc_auc_score for this epoch:  1.0
Validation loss for this epoch:  0.2861243724822998
Validation roc_auc_score for this epoch 0.9606405556627435
----------------EPOCH 4---------------------



Training loss for this epoch:  0.04653398245055643
Training roc_auc_score for this epoch:  1.0
Validation loss for this epoch:  0.37234835922718046
Validation roc_auc_score for this epoch 0.947134863978391
----------------EPOCH 5---------------------


...


Training loss for this epoch:  0.0856203149499682
Training roc_auc_score for this epoch:  1.0
Validation loss for this epoch:  0.44679841846227647
Validation roc_auc_score for this epoch 0.9309197651663405
----------------EPOCH 12---------------------



Training loss for this epoch:  0.06594485803765969
Training roc_auc_score for this epoch:  1.0
Validation loss for this epoch:  0.3613153174519539
Validation roc_auc_score for this epoch 0.9391389432485324
----------------EPOCH 13---------------------



Training loss for this epoch:  0.04916588565619248
Training roc_auc_score for this epoch:  1.0
Validation loss for this epoch:  0.3750198557972908
Validation roc_auc_score for this epoch 0.9348336594911938
----------------EPOCH 14---------------------



Training loss for this epoch:  0.0191635677862602
Training roc_auc_score for this epoch:  1.0
Validation loss for this epoch:  0.34013248458504675
Validation roc_auc_score for this epoch 0.9358121330724071
----------------EPOCH 15---------------------



Training loss for this epoch:  0.03128067001428766
Training roc_auc_score for this epoch:  1.0
Validation loss for this epoch:  0.3794193133711815
Validation roc_auc_score for this epoch 0.9356164383561644
best AUC_val:  0.9399217221135029


# **Predicting**

In [ ]:
test_transform = A.Compose([  A.Resize(height=input_shape[0], width=input_shape[1],p=1),
                              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                              albtorch.ToTensor(),
                               
                               
                            ])
test = pd.read_csv('/content/Test.csv')
test_dataset = ImageDataset('/content/test/',test,test_transform,mode='test')
test_data_loader = DataLoader(dataset=test_dataset,shuffle=False,batch_size=32)

In [ ]:
all_folds = []
for i in range(skf.n_splits):
  best_model = Net(NAME)
  best_model.load_state_dict(torch.load(f'best_model_{i}'))
  best_model.to(device)
  best_model.eval()
  final_outputs = []
  with torch.no_grad():
    tk0 = tqdm(test_data_loader, total=len(test_data_loader))
    for bi,d in enumerate(tk0):
      images = d['image']
      #send them to device 
      images = images.to(device,dtype=torch.float)
      if BOOL_CROP:
        bs, ncrops, c, h, w = images.size()
        outputs = best_model(images.view(-1, c, h, w)) # fuse batch size and ncrops
        outputs = outputs.view(bs, ncrops, -1).mean(1) # avg over crops
      else:
        outputs  = best_model(images)
        outputs = nn.Softmax()(outputs)[:,1]
      final_outputs.append(outputs.cpu().detach().numpy())
  final_outputs = np.concatenate(final_outputs)
  all_folds.append(final_outputs)

In [ ]:
ss = pd.read_csv('/content/Test.csv')

In [ ]:
ss['LABEL'] = np.mean(all_folds,axis=0)
ss = ss[['ID','LABEL']]
ss.head()

,ID,LABEL
0,GTWSHFYQ,0.053219
1,QTFSSMGD,0.000667
2,TBLBHSYT,0.999652
3,ZKETEOFG,0.814907
4,GKTPBGZP,0.682183


In [ ]:
ss.LABEL.max(),ss.LABEL.min()

(0.9999979138374329, 0.00024776466307230294)

In [ ]:
ss.to_csv(f'{NAME}_highQ_floats_{input_shape}_{skf.n_splits}_folds.csv',index=False)